# Creating Satellite Image Timelapses

For this demo, I will create a timelapse of satellite images using [Microsoft Planetary Computer](https://planetarycomputer.microsoft.com/). 

This code is based of the work from __Milan Janosov__. Check his [article](https://towardsdatascience.com/creating-satellite-image-timelapses-2b479f86ff52) out on this topic where he uses the new ESA Sentinel Hub API to make some really cool timelapses.



# Getting Started



MPC hosts datasets using the [STAC](https://stacspec.org/en) (SpatioTemporal Asset Catalog) specification.

 To interact with these datasets, we need to install the [pystac-client](https://pypi.org/project/pystac-client/) and [Planetary Computer SDK](https://pypi.org/project/planetary-computer/) via __pip__

In [1]:
import sys
!{sys.executable} -m pip install --quiet pystac-client planetary-computer --upgrade 

c:\Users\noah\AppData\Local\Programs\Python\Python312\python.exe: No module named pip


Next, we need to create a `pystac_client.Client` to access the data we want to pull from the MPC

To do this (and other downstream operations), I can use some code from the [MPC Docs](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/)


In [2]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Searching

Now we can use the STAC API to search for our assets that meet a certian criteria. The criterion in this case is a area we want to anaylze along with a date range.

Here, I will look at __Seattle, Washington__ during Summer 2023. 


In [4]:

time_range = "2023-06-21/2023-09-23" #date range
bbox = [-122.2751, 47.5469, -121.9613, 47.7458] #area

search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

39

39 `items` matched our search

`items` has a datatype of `'pystac.item_collection.ItemCollection'` 

In [5]:
print(type(items)) 

<class 'pystac.item_collection.ItemCollection'>


All that means is that this data type can be treated as an _Item Collection_.

_Item Collections_ hold the info we need to see some pictures!

Lets select an item and plot it using [IPython](https://ipython.readthedocs.io/en/stable/index.html)

In [15]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
print(selected_item)


<Item id=S2B_MSIL2A_20230815T185919_R013_T10TET_20230816T001753>


In [16]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

# Downloads

Now that we have our _item collection_, we need to __download__ imagery data to our local machine. 

Milan used bands 2 (red), 3 (green) , and 4 (blue) for his analysis.

To keep my analysis simpler, we will just extract the rendered preview. This _should_ be sufficient for the purposes of the timelapse.


In [17]:
#Here I will collect the hrefs of the rendered previews into a list

href_list = []
for item in items:
  href_list.append(item.assets["rendered_preview"].href)

In [18]:
len(href_list) #we have 11 file for our timelapse

39

Then using the pillow [Image Module]((https://pillow.readthedocs.io/en/stable/reference/Image.html)), I will save the images to my local machine

In [19]:
import sys
!{sys.executable} -m pip install --quiet pillow

c:\Users\noah\AppData\Local\Programs\Python\Python312\python.exe: No module named pip


In [20]:
import os
foldout = 'pictures'
if not os.path.exists(foldout):
    os.makedirs(foldout)

In [21]:
import requests
from PIL import Image
from io import BytesIO
for idx,href in enumerate(href_list):
    resp = requests.get(href)
    resp.raise_for_status()
    sio = BytesIO(resp.content)  # Create an in-memory stream of the content
    img = Image.open(sio)  # And load it
    img.save(f"pictures/picture_{idx}.png", "PNG")

Using some help from Milan's code again, lets assemble the frames into an animation

In [22]:
png_files = [f for f in os.listdir(foldout) if f.endswith('.png')]

png_files.sort()

frames = []

for png_file in png_files:
    file_path = os.path.join(foldout, png_file)
    img = Image.open(file_path)
    frames.append(img)


output_gif_path = 'footage_complete.gif'

frames[0].save(
    output_gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=200,  # Set the duration between frames in milliseconds
    loop=0  # Set loop to 0 for an infinite loop, or any positive integer for a finite loop
)

And see our work!


<img src='footage_complete.gif' width="750" align="center">

# Conclusion

This was a blast to work on. Special thanks to:

1. Milan for inspiring me on this.
1. The MPC team for hosting some pretty cool data

Im still pretty new to the geospatial world, but am finding myself falling in love with it. Excited to see what the future holds!


 -Noah